# Forecast Performance Scorecard

This scorecard provides a systematic, top-down approach to analyze forecast performance and identify areas for model improvement. The process follows a drill-down methodology from high-level business metrics to individual product analysis.

## Scorecard Process Flow:

### 1. **Portfolio Overview** 
Start with the big picture - how is the entire business performing against forecasts? This gives you the overall health of your forecasting system and identifies if there are systematic issues affecting the whole portfolio.

### 2. **Category Performance Analysis**
Drill down into sales categories (Very Low, Low, Alive, Medium, Winning, High Winning) to identify which product segments are driving forecast errors. This helps prioritize which categories need immediate attention.

### 3. **Error Distribution Deep Dive**
For problematic categories, analyze the distribution of forecast errors to understand:
- Are we consistently over-forecasting or under-forecasting?
- What's the magnitude of errors?
- Which products are the biggest contributors to poor performance?

### 4. **Individual Product Investigation**
Focus on the worst-performing products to understand root causes:
- Are there data quality issues?
- Is the product experiencing unusual patterns (promotions, stockouts, seasonality)?
- Are there external factors affecting demand?

### 5. **Actionable Insights Generation**
Based on the analysis, identify specific actions:
- Which products need model retraining?
- What data quality issues need fixing?
- Which categories need different forecasting approaches?
- What external factors should be incorporated into the model?

## How to Use This Scorecard:

1. **Run the complete notebook** to get the full analysis
2. **Start with the business-level view** to understand overall performance
3. **Identify problematic categories** using the error distribution plots
4. **Drill down into specific products** that are driving poor performance
5. **Document findings** walk through lots of skus and identofy dominnat issues then correct. Don't focus on correcting specific edge cases. This will end up messing working cases



# Scorecard

In [25]:
# =============================================================================
# ALL IMPORTS - CONSOLIDATED IN FIRST CELL
# =============================================================================

# Standard library imports
import sys
import importlib
from importlib import reload

# Third-party imports
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window

# Local imports
import configuration as CONFIG
from functions_library.supabase_connection import SupabaseClient
import functions_library.forecast_evaluation as ED

# =============================================================================
# MODULE RELOADING - Ensure latest versions are loaded
# =============================================================================
importlib.reload(CONFIG)
importlib.reload(sys.modules['functions_library.supabase_connection'])
importlib.reload(sys.modules['functions_library.forecast_evaluation'])

# =============================================================================
# SPARK INITIALIZATION
# =============================================================================
spark = SparkSession.builder.appName('ForecastEvaluation').master('local[1]').getOrCreate()
spark.sparkContext.setLogLevel("WARN")

# =============================================================================
# SUPABASE CLIENT INITIALIZATION
# =============================================================================
storage_client = SupabaseClient()

In [47]:
EVALUATION_START_DATE = "2025-10-01"
EVALUATION_END_DATE = "2025-10-07"
RUN_ID = 3

In [48]:
products = storage_client.batch_read_table("products", spark)
prod_day_history = storage_client.download_processed_features(spark)
prod_day_forecast = storage_client.download_forecast_run(RUN_ID, spark)


In [49]:
prod_day_history.count()

185931

In [50]:
prod_day_history.select(F.max("date")).show()

+----------+
| max(date)|
+----------+
|2025-10-14|
+----------+



In [51]:
prod_day_forecast.show()

+--------------------+--------------------+------------+--------------+------+---------------+----------+-------------------+
|          product_id|  age_sales_category|age_category|sales_category| style|forecast_method|      date|           forecast|
+--------------------+--------------------+------------+--------------+------+---------------+----------+-------------------+
|03362d4e-d87f-4f5...|04| Mature_04| Ma...|  04| Mature|       03| Low|MW1653|        croston|2025-10-01|0.40271186440677975|
|03362d4e-d87f-4f5...|04| Mature_04| Ma...|  04| Mature|       03| Low|MW1653|        croston|2025-10-02|0.40271186440677975|
|03362d4e-d87f-4f5...|04| Mature_04| Ma...|  04| Mature|       03| Low|MW1653|        croston|2025-10-03|0.40271186440677975|
|03362d4e-d87f-4f5...|04| Mature_04| Ma...|  04| Mature|       03| Low|MW1653|        croston|2025-10-04|0.40271186440677975|
|03362d4e-d87f-4f5...|04| Mature_04| Ma...|  04| Mature|       03| Low|MW1653|        croston|2025-10-05|0.40271186440

In [52]:
evaluation_data = (prod_day_forecast
                   .join(prod_day_history.select("product_id", "date", "sales_units"), on=["product_id", "date"], how="inner")
                   .groupBy("product_id", "style", "age_sales_category", "age_category", "sales_category", "forecast_method")
                   .agg(F.sum("sales_units").alias("sales_units"),
                        F.sum("forecast").alias("forecast")))
evaluation_data = ED.calculate_bias_and_error_bins(evaluation_data)

In [53]:
validation_data = (prod_day_history
                   .join(prod_day_forecast.select("product_id", "date", "forecast_method", "forecast"), on=["product_id", "date"],how="left"))
validation_data = ED.calculate_bias_and_error_bins(validation_data)

# Business Summary

In [70]:
reload(ED)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [71]:
pdf, plot = ED.plot_business_level(validation_data, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()

Evaluation frame (2025-10-01 - 2025-10-07):
  Total Sales: 3982
  Total Forecast: 4501.270459932322
  Over/Under Forecast: 519.2704599323224 units (13.0%)


# Overall Evaluation

In [55]:
# Define per-bin good thresholds (understock / overstock)
GOOD_THRESHOLD = {
    "02| Very Low": (5, 10),
    "03| Low": (5, 10),
    "04| Alive": (5, 20),
    "05| Medium": (5, 20),
    "06| Winning": (5, 20), # Bias
    "07| High Winning": (5, 30) # Bias
}

In [ ]:
df, plot =ED.plot_attribute_distribution(evaluation_data, ["sales_category"])
plot.show()


In [68]:
reload(ED)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [58]:
ED.plot_accuracy_distribution(evaluation_data, method="bias").show()
ED.plot_accuracy_distribution(evaluation_data, method="error").show()

In [59]:
df = (evaluation_data
    .where(~(F.col("sales_category").isin(["02| Very Low", "03| Low", "07| High Winning"]))))
ED.plot_accuracy_distribution(df, method="bias").show()

# Investigate Sales Categories

In [60]:
SELECTED_SALES_BIN = "07| High Winning" 
METHOD = "bias"

In [61]:
evaluation_data_category = evaluation_data.where(F.col("sales_category") == SELECTED_SALES_BIN)
ED.analyze_category_performance(evaluation_data_category)

,product_count,sales_units,forecast,error
0,7,1673,1861.726217,188.726217


In [62]:
ED.plot_accuracy_distribution(evaluation_data_category, method=METHOD).show()

# Investigate Products

In [63]:
SELECTED_BIAS_BIN = "22| > +50"
LIMIT = 10

In [64]:
(evaluation_data_category
    .where(F.col("sales_category") == SELECTED_SALES_BIN)
    .where(F.col("bias_bin").rlike(SELECTED_BIAS_BIN))
    .orderBy(F.desc("error")).limit(LIMIT).toPandas())

,product_id,style,age_sales_category,age_category,sales_category,forecast_method,sales_units,forecast,bias,error,bias_bin,error_bin
0,053dacb8-52a2-41ce-9e39-f2197e2d2760,None,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,101,240.078787,1.377018,139.078787,22| > +100,22| > +50


In [74]:
reload(ED)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [75]:
# Example usage
SELECTED_STYLE = "MW1341" 
pdf_product, plot = ED.plot_product_details_daily_short(validation_data, SELECTED_STYLE, products_df=products, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()


Product Title: MW1341 -  عباية بليزر بتطريز اسود
Price: 260.87
Status: ACTIVE
